In [59]:
from IPython.display import Image
import numpy as np
import scipy as sp

- 无约束优化问题
- GD/SGD：广泛应用在深度神经网络的求解中
    - 如果把神经网络视为一个经典的优化问题的话，那么所有的weights/parameters就可以被认为是优化问题中的决策变量（decision variables）
- CG，Newton, Quasi-Newton (BFGS 等)：
    - 经典优化问题
    - xgboost：引入二阶导数信息(hessian)计算 split gain
      $$
      \text{Gain} = \frac{1}{2} \left[ \frac{(\sum_{i \in L} g_i)^2}{\sum_{i \in L} h_i + \lambda} + \frac{(\sum_{i \in R} g_i)^2}{\sum_{i \in R} h_i + \lambda} - \frac{(\sum_{i \in N} g_i)^2}{\sum_{i \in N} h_i + \lambda} \right] - \gamma
      $$
    - trpo/ppo

## gradient descent

In [6]:
Image(url='https://miro.medium.com/v2/resize:fit:1400/format:webp/1*xcUuDFPa7DIQxxHKatXIYA.png', width=500)

## Conjugate Gradient

$$
Ax=b
$$
- $A$ 对称正定矩阵
- 在优化问题中，这等同于求解形式为
  $$
  \nabla^2f(x)\cdot p=-\nabla f(x)
  $$

In [71]:
Image(url='../../imgs/cg.png', width=500)

In [81]:
A = np.array([[4, 0, 1, 0],
              [0, 5, 0, 0],
              [1, 0, 3, 2],
              [0, 0, 2, 4]])
b = np.array([-1, -0.5, -1, 2])
x, exit_code = sp.sparse.linalg.cg(A, b)

In [82]:
x

array([ 6.04404796e-18, -1.00000000e-01, -1.00000000e+00,  1.00000000e+00])

In [83]:
A.dot(x)

array([-1. , -0.5, -1. ,  2. ])

## newton's methods

- 没有学习率的概念
    - hessian 矩阵的逆提供了步长的调整
- quadratic approximations

In [7]:
Image(url='https://miro.medium.com/v2/resize:fit:1400/format:webp/1*AHxdnlJjQZNzDZp-reRIzg.png', width=500)

In [8]:
Image(url='https://miro.medium.com/v2/resize:fit:1400/format:webp/1*6Nb-j3wx7Yrn265ITzPJ3w.png', width=500)

In [9]:
Image(url='https://miro.medium.com/v2/resize:fit:1400/format:webp/1*qJX12tZwVh9YdGfx8e34iQ.png', width=500)

In [10]:
Image(url='https://miro.medium.com/v2/resize:fit:1400/format:webp/1*rmcorcWtNbMB65LyWXt5-w.png', width=500)

### hessian matrix

- 当 $f(\mathbf x)$ 是多元（multi-variables）函数

In [11]:
Image(url='https://miro.medium.com/v2/resize:fit:1400/format:webp/1*ieKysCeXqBfwosXEmJSvtw.png', width=500)

In [12]:
Image(url='https://miro.medium.com/v2/resize:fit:1400/format:webp/1*n-EYGjoYVVc7DlD8KjPx7A.png', width=500)

In [22]:
# Function definition
def f(x):
    return (x - 2)**2

# Derivative of the function
def df(x):
    return 2 * (x - 2)

# Second derivative of the function
def ddf(x):
    return 2

# Gradient descent method
def gradient_descent(x0, learning_rate, iterations):
    x = x0
    trajectory = [x]
    for _ in range(iterations):
        x -= learning_rate * df(x)
        trajectory.append(x)
    return trajectory

# Newton's method
def newtons_method(x0, iterations):
    x = x0
    trajectory = [x]
    for _ in range(iterations):
        x -= df(x) / ddf(x)
        trajectory.append(x)
    return trajectory

# Parameters setup
x0 = 0             # Initial value
learning_rate = 0.2  # Learning rate
iterations = 10      # Number of iterations

# Perform iterations
gd_trajectory = gradient_descent(x0, learning_rate, iterations)
newton_trajectory = newtons_method(x0, iterations)

gd_trajectory, newton_trajectory

([0,
  0.8,
  1.28,
  1.568,
  1.7408000000000001,
  1.8444800000000001,
  1.9066880000000002,
  1.9440128,
  1.96640768,
  1.979844608,
  1.9879067648],
 [0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0])

### rosen

$$
f(x,y)=(x-1)^2 + 10(y-x^2)^2
$$

In [6]:
import sympy as sy

In [13]:
x, y = sy.symbols('x y')

In [12]:
f = (x-1)**2 + 10*(y-x**2)**2

In [14]:
f = sy.Matrix([f, ])

In [15]:
f.jacobian(sy.Matrix([x, y]))

Matrix([[-40*x*(-x**2 + y) + 2*x - 2, -20*x**2 + 20*y]])

In [16]:
sy.hessian(f, [x, y])

Matrix([
[120*x**2 - 40*y + 2, -40*x],
[              -40*x,    20]])

In [17]:
# Rosenbrock function definition
def rosenbrock(x, y):
    return (x - 1)**2 + 10 * (y - x**2)**2

# Gradient of the Rosenbrock function
def rosenbrock_grad(x, y):
    dx = 2 * (x - 1) - 40 * x * (y - x**2)
    dy = 20 * (y - x**2)
    return np.array([dx, dy])

# Hessian matrix of the Rosenbrock function
def rosenbrock_hessian(x, y):
    hxx = -40 * (y - 3*x**2) + 2
    hxy = -40 * x
    hyx = -40 * x
    hyy = 20
    return np.array([[hxx, hxy], [hyx, hyy]])

In [23]:
# Gradient descent method for a multivariable function with corrected data types
def gradient_descent_multi(x0, y0, learning_rate, iterations):
    point = np.array([x0, y0], dtype=float)  # Ensure point is floating point
    trajectory = [point.copy()]
    for _ in range(iterations):
        grad = rosenbrock_grad(point[0], point[1])
        point -= learning_rate * grad
        trajectory.append(point.copy())
    return trajectory

# Newton's method for a multivariable function with corrected data types
def newtons_method_multi(x0, y0, iterations):
    point = np.array([x0, y0], dtype=float)  # Ensure point is floating point
    trajectory = [point.copy()]
    for _ in range(iterations):
        grad = rosenbrock_grad(point[0], point[1])
        hessian = rosenbrock_hessian(point[0], point[1])
        point -= np.linalg.inv(hessian).dot(grad)
        trajectory.append(point.copy())
    return trajectory

In [57]:
x0, y0 = 0, 0  # Initial values
learning_rate = 0.05  # Learning rate for gradient descent
iterations = 10      # Number of iterations

# Perform iterations with corrected data types
gd_trajectory = gradient_descent_multi(x0, y0, learning_rate, iterations)
newton_trajectory = newtons_method_multi(x0, y0, iterations)

In [58]:
gd_trajectory, newton_trajectory

([array([0., 0.]),
  array([0.1, 0. ]),
  array([0.188, 0.01 ]),
  array([0.25967066, 0.035344  ]),
  array([0.3170406 , 0.06742885]),
  array([0.3643574 , 0.10051474]),
  array([0.40442675, 0.13275631]),
  array([0.4390676, 0.163561 ]),
  array([0.46950229, 0.19278036]),
  array([0.49658667, 0.2204324 ]),
  array([0.52094071, 0.24659832])],
 [array([0., 0.]),
  array([1., 0.]),
  array([1., 1.]),
  array([1., 1.]),
  array([1., 1.]),
  array([1., 1.]),
  array([1., 1.]),
  array([1., 1.]),
  array([1., 1.]),
  array([1., 1.]),
  array([1., 1.])])

## Quasi-Newton methods

- 牛顿法的性能问题
    - hessian 矩阵（对称的）的计算 $\frac{n(n+1)}{2}(O(n^2))$，
        - $n$ 是多元变量函数的变量的数量；
    - 求逆是 $O(n^3)$ 的时间复杂度；
- 拟牛顿法的关键在于不需要计算hessian，更不需要计算其 inv；
- scipy.optimize
    - https://docs.scipy.org/doc/scipy/tutorial/optimize.html

In [70]:
Image(url='../../imgs/bfgs.png', width=500)

- $B_k$ 近似 $H^{-1}$ （inverse hessian）
    - $B$: positive definite
- approx newton direction: $s_k=x_{k+1}-x_k=-B_k^{-1}g_k$
- update $B_k$
    - $B_{k+1}=\text{update}(B_k, x^{k+1}-x^k, g^{k+1}-g^k)$

In [19]:
Image(url='../../imgs/bk_types.png', width=500)

In [69]:
Image(url='https://miro.medium.com/v2/resize:fit:1320/format:webp/1*UyPDBKccaqfRTc00l1rKEg.jpeg', width=400)

In [67]:
from scipy.optimize import rosen, rosen_der

In [66]:
x0 = [0, 0]
res = sp.optimize.minimize(rosen, x0, method='BFGS', jac=rosen_der, 
               options={'disp': True})

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 19
         Function evaluations: 24
         Gradient evaluations: 24


In [68]:
res.x

array([0.99999913, 0.99999825])